# Criação dicionário NRC, a partir da lista fornecida

NOTA:é necessario porque o modulo NRCLex não esta disponivel para portugues.
    -- Este ficheiro permitiu obter O ficheiro NTRC_portuguese.json que contem o dicionario de palavras em portugues ja tratado
Definição da função que avalia uma frase lematizada, usando um dicionário criado por nós com base no corpus fornecido pelo ``NRCLex`` em português.

In [2]:
import pandas as pd

In [7]:
#importar o dataset do NRCLex em português

filepath = 'NRCPortugues.txt'
emolex_df = pd.read_csv(filepath,   names=["wordEN", "word", "emotion", "emotion_intensity_score"], index_col=False, skiprows=45, sep='\t')
emolex_df.head(12)

,wordEN,word,emotion,emotion_intensity_score
0,torment,tormento,anger,0.885
1,wrath,ira,anger,0.885
2,vicious,vicioso,anger,0.884
3,massacre,massacre,anger,0.882
4,threatening,ameaçador,anger,0.882
5,pissoff,não me chateies,anger,0.875
6,abhorrent,repugnante,anger,0.875
7,bloodthirsty,sanguinário,anger,0.875
8,fighting,combate,anger,0.868
9,annihilated,aniquilado,anger,0.865


Criação do dicionário com as palavras do dataset + os sentimentos associados às mesmas

In [8]:
emolex_df = emolex_df.drop(columns = 'wordEN')

lex_dic = {}
x=0
for i in emolex_df['word']:
    lex_dic[i]=[]

for i,row in emolex_df.iterrows():
    
    lex_dic[row['word']].append( (row['emotion'], row['emotion_intensity_score']) )



Mapeamento dos sentimentos associados a cada palavra em **positivo** e **negativo**.

In [9]:
lp =['anticipation','joy', 'surprise', 'trust']
ln =['anger', 'disgust', 'fear', 'sadness']
dic = {}
for i in lex_dic:
    dic[i]='a'

for i in lex_dic:
    x=lex_dic[i]
    contp=0
    contn=0
    for k in x:
        (e,c)=k
        if e in lp:
            contp = contp + c
        elif e in ln:
            contn = contn + c
    if(contp > contn):
        dic[i]='positivo'
    else :
        dic[i]='negativo'



Guardar os dados lidos do ficheiro texto num dicionario, que classifica cada palavra num dado sentimento.

In [14]:
import json

file = open("NRC_portuguese.json", "w")
json.dump(dic,file)
file.close()

Ler o dicionario de palavras em Portugues

In [30]:
file = open("NRC_portuguese.json", "r")
NRC_file = file.read()
NRC = json.loads(NRC_file)
file.close()


In [32]:
type (NRC)

dict

## Para classificação do dataset

In [37]:
# Função que faz lematização, tokenização, retira pontuação e stop words, coloca o texto em minusculas
import spacy
def dataPrep(review):
    
    nlp = spacy.load("pt_core_news_sm")
    doc = nlp(review)
    res = " ".join([token.lemma_.lower() for token in doc if not token.is_stop and not token.is_punct])   
    
    return res



def avalia(review):
    
    d = {'positive': 0, 'negative': 0, 'fear': 0, 'anger' :0, 'antecipation': 0, 'surprise':0 , 'sadness':0, 'disgust':0, 'joy':0}
   
 
    for i in range(len(review)):
        #print(frase[i])
        word = review[i]

        ## Verificar se a word esta no corpus
        if word in NRC:
          emotion = NRC[word]
          d[emotion] = d[emotion] + 1
         
    ## Retorna o dicionario com as emoções ordenado
    return {k: d[k] for k in sorted(d)}
    


texto = "Não deixe que as pessoas te façam desistir daquilo que você mais quer na vida. Acredita. Lutar. Conquistar. E acima de tudo, se feliz!"
res = dataPrep(texto)
print(res)
avalia(res.split(" "))




OSError: [E050] Can't find model 'pt_core_news_sm'. It doesn't seem to be a Python package or a valid path to a data directory.

# Classificação do Dataset

Estamos a assumir que os negativos (0) vão despoletar reação e que os positivos (1) não vão despoletar reação.